In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

stock_market_prediction_and_sentimental_analysis_path = kagglehub.competition_download('stock-market-prediction-and-sentimental-analysis')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stock-market-prediction-and-sentimental-analysis/sample_submission.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Combined_News_DJIA(train).csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_DJIA_Table.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/TEST_Redit_news.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/RedditNews(train).csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_Combined_News.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/DJIA_table(train).csv


In [ ]:
cntr=pd.read_csv("/kaggle/input/stock-market-prediction-and-sentimental-analysis/Combined_News_DJIA(train).csv")
nrntr=pd.read_csv("/kaggle/input/stock-market-prediction-and-sentimental-analysis/RedditNews(train).csv")
djtr=pd.read_csv("/kaggle/input/stock-market-prediction-and-sentimental-analysis/DJIA_table(train).csv")
cnts=pd.read_csv("/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_Combined_News.csv")
nrnts=pd.read_csv("/kaggle/input/stock-market-prediction-and-sentimental-analysis/TEST_Redit_news.csv")
djts=pd.read_csv("/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_DJIA_Table.csv")


In [ ]:
import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from textblob import TextBlob

from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [ ]:
li = [cntr, djtr, nrntr, nrnts, cnts, djts]
for l in li:
    print(l.shape)
    print(l.isnull().sum())


(1989, 27)
Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64
(1863, 7)
Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
dtype: int64
(73608, 2)
Date    0
News    0
dtype: int64
(4551, 2)
Date    0
News    0
dtype: int64
(126, 27)
Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    0
Top24    0
Top25    0
dtype: int64
(126, 5)
Date      0
Open      0
High      0
Low       0
Volume    0
dtype: int64


In [ ]:
cntr[["Top23", "Top24", "Top25"]] = cntr[["Top23", "Top24", "Top25"]].fillna("no_news")


In [ ]:
nrntr["news_no"] = nrntr.groupby("Date").cumcount() + 1
nrntr = nrntr[nrntr["news_no"] <= 25]

wide_nrntr = nrntr.pivot(index="Date", columns="news_no", values="News")
wide_nrntr.columns = [f"Top{i}" for i in wide_nrntr.columns]

wide_nrntr = wide_nrntr.fillna("no_news").reset_index()
wide_nrntr["label"] = 0


In [ ]:
nrnts["news_no"] = nrnts.groupby("Date").cumcount() + 1
nrnts = nrnts[nrnts["news_no"] <= 25]
print(nrnts)
wide_nrnts = nrnts.pivot(index="Date", columns="news_no", values="News")
wide_nrnts.columns = [f"Top{i}" for i in wide_nrnts.columns]

wide_nrnts = wide_nrnts.fillna("no_news").reset_index()
wide_nrnts["label"] = 0
print(wide_nrnts)

          Date                                               News  news_no
0     7/1/2016  A 117-year-old woman in Mexico City finally re...        1
1     7/1/2016   IMF chief backs Athens as permanent Olympic host        2
2     7/1/2016  The president of France says if Brexit won, so...        3
3     7/1/2016  British Man Who Must Give Police 24 Hours' Not...        4
4     7/1/2016  100+ Nobel laureates urge Greenpeace to stop o...        5
...        ...                                                ...      ...
4545  1/1/2016  Russian President Vladimir Putin has endorsed ...       21
4546  1/1/2016  Kim Jong-un Talks of Reconciliation in New Yea...       22
4547  1/1/2016             Terror bombing at Eilat hotel thwarted       23
4548  1/1/2016  Remains of over 4,500-yrs-old shepherd's hut f...       24
4549  1/1/2016  Voice of the Caliphate radio broadcasts anti-T...       25

[4550 rows x 3 columns]
          Date                                               Top1  \
0     

In [ ]:
cntr = cntr.drop(columns=["Label"])
cnts = cnts.drop(columns=["Label"])

wide_nrntr = wide_nrntr.drop(columns=["label"])
wide_nrnts = wide_nrnts.drop(columns=["label"])


In [ ]:
cntr["Date"] = pd.to_datetime(cntr["Date"], dayfirst=True,format='mixed', errors='coerce')
wide_nrntr["Date"] = pd.to_datetime(wide_nrntr["Date"], dayfirst=True, format='mixed', errors='coerce')


In [ ]:
cnts["Date"] = pd.to_datetime(cnts["Date"], format="mixed", dayfirst=True, errors='coerce')
wide_nrnts["Date"] = pd.to_datetime(wide_nrnts["Date"], format="mixed", dayfirst=True,  errors='coerce')


In [ ]:
final_df = pd.merge(cntr, wide_nrntr, on="Date", how="outer", suffixes=("_cntr", "_nrntr"))

for i in range(1, 26):
    final_df[f"Top{i}"] = final_df[f"Top{i}_cntr"].combine_first(
        final_df[f"Top{i}_nrntr"]
    )

final_df.drop(columns=[c for c in final_df.columns if "_cntr" in c or "_nrntr" in c], inplace=True)
fdf = final_df.sort_values("Date").reset_index(drop=True)


In [ ]:
final_dft = pd.merge(cnts, wide_nrnts, on="Date", how="outer", suffixes=("_cnts", "_nrnts"))

for i in range(1, 26):
    final_dft[f"Top{i}"] = final_dft[f"Top{i}_cnts"].combine_first(
        final_dft[f"Top{i}_nrnts"]
    )

final_dft.drop(columns=[c for c in final_dft.columns if "_cnts" in c or "_nrnts" in c], inplace=True)
fdft = final_dft.sort_values("Date").reset_index(drop=True)


In [ ]:
stop_words = set(ENGLISH_STOP_WORDS)
stemmer = PorterStemmer()

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text[1:-1]
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words]
    return " ".join(words)

def clean_textt(text):
    if pd.isna(text):
        return ""
    text = text[0:-1]
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words]
    return " ".join(words)


In [ ]:
top_cols = [f"Top{i}" for i in range(1, 26)]

for col in top_cols:
    fdf[col] = fdf[col].astype(str).apply(clean_text)
    fdft[col] = fdft[col].astype(str).apply(clean_textt)

fdf["com_n"]  = fdf[top_cols].agg(" ".join, axis=1)
fdft["com_n"] = fdft[top_cols].agg(" ".join, axis=1)


In [ ]:
def get_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity, blob.sentiment.subjectivity

fdf[["polarity", "subjectivity"]] = fdf["com_n"].apply(lambda x: pd.Series(get_sentiment(x)))
fdft[["polarity", "subjectivity"]] = fdft["com_n"].apply(lambda x: pd.Series(get_sentiment(x)))


In [ ]:
djtr["Date"] = pd.to_datetime(djtr["Date"], format="%d-%m-%Y")
djts["Date"] = pd.to_datetime(djts["Date"], format="%d-%m-%Y")

mdf  = pd.merge(fdf, djtr, on="Date", how="inner")
mdft = pd.merge(fdft, djts, on="Date", how="inner")
mdft.head()

,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top23,Top24,Top25,com_n,polarity,subjectivity,Open,High,Low,Volume
0,2016-01-04,saudi arabia break tie iran amid row saudi exe...,exclus uk govern urg reveal role get saudi ara...,china halt trade stock fall,earthquak measur magnitud strike indi,viru link shrink newborn brain spread rapidli ...,new photo spacex booster sooti undamag rock,indian consul afghanistan attack gunm,iran call major haj,denmark respond swedish border check control e...,...,aftermath muslim cleric s execut saudi arabia ...,iran arrest attack saudi embass,oil price jump pct saudi arabia iran tension,saudi arabia break tie iran amid row saudi exe...,-0.018959,0.211209,17405.48047,17405.48047,16957.63086,148060000
1,2016-01-05,dutch govern back strong encrypt condemn backdoor,isi milit strap bomb year old boy blow kill ch...,canada proceed controversi billion saudi arm d...,brazil head worst recess,british isi milit s desper plea doctor join te...,malaysia s school start hold anti isi talk kid,saudi arabia omit uk death penalti strategi sa...,isi execut femal journalist post complain limi...,u s justic depart su volkswagen billion allege...,...,known hacker caus power outag signal troubl es...,kuwait latest saudi alli downgrad tie ira,car emit oxygen indian suprem court car manufa...,dutch govern back strong encrypt condemn backd...,0.072846,0.408882,17147.50000,17195.83984,17038.60938,105750000
2,2016-01-06,china shut stock market min loss,arm suspect shot dead tri storm pari polic statio,oil price fall barrel fresh year lo,german outrag mayor advic women raft harassmen,german public broadcast zdf apologis delay rep...,iran accus saudi bomb embassi sana yeme,protest demand canada cancel b saudi contrac,snooper s charter cost british live mp warn ns...,islam scholar malaysia indonesia issu fatwa po...,...,work hospit central china demolish forc morn d...,pari close champ elyse car sunday mont,transcanada corp file billion lawsuit u s gove...,china shut stock market min loss arm suspect s...,-0.008165,0.295602,17154.83008,17154.83008,16817.61914,120250000
3,2016-01-07,report sexual assault women european citi incl...,shelter forcibl convert children christian rai...,iran ban citizen make pilgrimag mecc,gaddafi warn blair oust open door jihadi want ...,turkish firm sell fake life vest soak water re...,german minist say deport possibl cologn sexual...,mao statu china pull author ft gold giant mock...,poacher hunt dog south africa award medal hero...,polic use terror power spi offic blow whistl r...,...,north korea claim test h bomb justic,uk dwp told woman ill benefit day die,venezuela s new parliament remov pictur hugo c...,report sexual assault women european citi incl...,0.001659,0.502445,16888.35938,16888.35938,16463.63086,176240000
4,2016-01-08,el chapo taken prison escap month ag,el chapo caugh,children abus cathol boy choir run pope benedi...,north korea defend latest nuclear test say fat...,group unit nation expert warn chill effect fre...,gunman open hotel popular western tourist hurg...,french propos strip citizenship terror set alarm,hungari support eu sanction poland say hungari...,insid delhi s gadget graveyard west s e wast e...,...,seychel debt restructur climat adapt swap near...,german video liken mao hitler china want punish,cereal harvest world fall yr impact drought st...,el chapo taken prison escap month ag el chapo ...,0.082671,0.284012,16519.16992,16651.89063,16314.57031,141850000


In [ ]:
mdf["Ch"] = (mdf["Close"] - mdf["Open"]) * 100 / mdf["Open"]
mdf["Label"] = np.where(mdf["Ch"] >= 0, 1, 0)
mdf = mdf.sort_values("Date").reset_index(drop=True)
mdft = mdft.sort_values("Date").reset_index(drop=True)

In [ ]:
print(mdf["Ch"].unique())

[ 2.64370272  0.44911485 -1.18175193 ...  0.98938666 -0.61015515
 -0.9415841 ]


In [ ]:
import numpy as np

mdf = mdf.sort_values("Date").reset_index(drop=True)

# Previous prices (memory of market)
mdf["prev_close"] = mdf["Close"].shift(1)
mdf["prev_open"]  = mdf["Open"].shift(1)

# Returns (price movement strength)
mdf["return_1d"] = (mdf["Close"] - mdf["prev_close"]) / mdf["prev_close"]
mdf["return_3d"] = mdf["Close"].pct_change(3)
mdf["return_7d"] = mdf["Close"].pct_change(7)

# Rolling averages (trend direction)
mdf["ma_3"] = mdf["Close"].rolling(3).mean()
mdf["ma_7"] = mdf["Close"].rolling(7).mean()

# Volatility (risk level)
mdf["volatility_5"] = mdf["Close"].rolling(5).std()

# Price range features (intraday strength)
mdf["high_low_diff"] = mdf["High"] - mdf["Low"]
mdf["open_close_diff"] = mdf["Close"] - mdf["Open"]

# Volume momentum
mdf["volume_change"] = mdf["Volume"].pct_change()

# Sentiment strength
mdf["sentiment_strength"] = mdf["polarity"] * mdf["subjectivity"]

# Sentiment momentum
mdf["sentiment_change"] = mdf["polarity"].diff()

# Drop NaN rows created by rolling/shift
mdf = mdf.dropna().reset_index(drop=True)


In [ ]:
print(mdf.columns)

Index(['Date', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7', 'Top8',
       'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15', 'Top16',
       'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23', 'Top24',
       'Top25', 'com_n', 'polarity', 'subjectivity', 'Open', 'High', 'Low',
       'Close', 'Volume', 'Adj Close', 'Ch', 'Label', 'prev_close',
       'prev_open', 'return_1d', 'return_3d', 'return_7d', 'ma_3', 'ma_7',
       'volatility_5', 'high_low_diff', 'open_close_diff', 'volume_change',
       'sentiment_strength', 'sentiment_change'],
      dtype='object')


In [ ]:
features = ['Open', 'High', 'Low', 'Volume',
            'polarity', 'subjectivity','prev_close','Ch']

X = mdf[features]
y = mdf['Close']


In [ ]:
split = int(len(mdf) * 0.8)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=500,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, min_samples_split=5, n_estimators=500,
                      n_jobs=-1, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=4,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [ ]:
y_pred = model.predict(X_test)



In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R² Score:", r2)
print("RMSE:", rmse)


R² Score: -0.41025825422010787
RMSE: 671.7859250102023


In [ ]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=6.12681e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [ ]:
mdf["return"] = mdf["Close"].pct_change()


In [ ]:
mdf["t"] = np.arange(len(mdf))


In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def sine_func(t, A, B, C, D):
    return A * np.sin(B * t + C) + D


In [ ]:
y = mdf["return"].fillna(0).values
t = mdf["t"].values

params, _ = curve_fit(sine_func, t, y, maxfev=10000)
A, B, C, D = params

print("Fitted Parameters:", params)


Fitted Parameters: [6.88899397e-04 9.98588693e-01 1.93037164e+00 3.13326565e-04]


In [ ]:
mdf["sine_pred"] = sine_func(t, A, B, C, D)


In [ ]:
mdf["sin_5"] = np.sin(2*np.pi*mdf["t"]/5)
mdf["cos_5"] = np.cos(2*np.pi*mdf["t"]/5)
mdf["sin_20"] = np.sin(2*np.pi*mdf["t"]/20)
mdf["cos_20"] = np.cos(2*np.pi*mdf["t"]/20)


In [ ]:
features = ["sine_pred", "polarity", "subjectivity", "Open", "Volume"]
X = mdf[features]
y = mdf["return"]


In [ ]:
features = [
    "sine_pred",      # cycle pattern
    "polarity",       # news sentiment
    "subjectivity",
    "Open",
    "High",
    "Low",
    "Volume",
    "sin_5", "cos_5", # short cycle
    "sin_20", "cos_20" # longer cycle
]

X = mdf[features].fillna(0)
y = mdf["return"].fillna(0)


In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)


In [ ]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=6.12681e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbm_model = GradientBoostingRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    random_state=42
)

gbm_model.fit(X_train, y_train)
y_pred_gbm = gbm_model.predict(X_test)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

def evaluate(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{name} → R²: {r2:.4f}, RMSE: {rmse:.6f}")

evaluate("XGBoost", y_test, y_pred_xgb)
evaluate("Ridge", y_test, y_pred_ridge)
evaluate("GBM", y_test, y_pred_gbm)


XGBoost → R²: -0.5642, RMSE: 707.499963
Ridge → R²: 0.9918, RMSE: 51.147355
GBM → R²: -0.4266, RMSE: 675.670865


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import numpy as np

mdft = mdft.sort_values("Date").reset_index(drop=True)
mdft["t"] = np.arange(len(mdft)) + len(mdf)

A, B, C, D = 6.88899397e-04, 9.98588693e-01, 1.93037164e+00, 3.13326565e-04

mdft["sine_pred"] = A * np.sin(B * mdft["t"] + C) + D
mdft["sin_5"] = np.sin(2*np.pi*mdft["t"]/5)
mdft["cos_5"] = np.cos(2*np.pi*mdft["t"]/5)

features = [
    "Open", "High", "Low", "Volume",
    "polarity", "subjectivity",
    "sine_pred", "sin_5"  # drop cos_5 to reduce collinearity
]

# Training data from mdf
X_train = mdf[features].fillna(0)
y_train = mdf["return"].fillna(0)

# New data
X_new = mdft[features].fillna(0)

# ---------- Scaling ----------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_new_scaled = scaler.transform(X_new)

# ---------- Ridge ----------
ridge_model = Ridge(alpha=0.1)   # strong regularization
ridge_model.fit(X_train_scaled, y_train)

# ---------- Prediction ----------
pred_returns = ridge_model.predict(X_new_scaled)
pred_returns1 = pred_returns * 100
pred_returns1 = np.round(pred_returns1, 6)

print(pred_returns1)


[-2.686812 -0.348162 -2.038146 -2.580232 -0.342958 -0.076783  0.58562
 -1.959709  1.678481 -3.114204  0.419821 -3.24095   1.450138  1.447225
 -1.261236  1.950993 -1.318277  0.365972  2.528354 -0.53933  -1.881711
 -0.044059  0.652319 -1.700551 -2.070525  0.129298  0.304869 -2.381261
  1.895735  1.218771  1.801555 -0.374221 -0.817232  1.664683 -1.262531
 -0.931041  1.029855  0.027933 -0.123851  2.168868 -0.221153 -0.16231
  0.498715  0.411105 -0.630319  0.37484  -0.28505   1.35538   0.155111
 -0.366551  0.589444  1.250881  0.846454  0.113553 -0.097433 -0.644616
 -0.308515  0.186994  0.405024  0.896807 -0.060664  0.425082 -0.495295
 -0.849966  0.397414 -1.250475  0.746194  0.770863  1.051888  1.171808
  0.150923 -0.320138  0.525563  0.423955  0.535683 -0.702083 -0.201461
 -0.835766  0.018502  0.107493 -1.303793 -0.98106   0.7974   -1.214246
 -0.768332  0.147328  0.192509 -0.198171  1.414858 -1.260672  0.033906
 -1.067141  1.499432 -1.40557   0.420156 -1.118782  0.866531  0.090041
  1.4534

In [ ]:
prev_close = float(mdf["Close"].iloc[-1])
pred_close=[]


for r in pred_returns:
    c = prev_close * (1.0 + r)

    pred_close.append(c)
    prev_close = c

mdft["Predicted_Close"] = pred_close
mdft["Predicted_Close"]

0      16956.851557
1      16897.814172
2      16553.411982
3      16126.295508
4      16070.989024
           ...     
121    15653.730284
122    15881.641512
123    16145.262072
124    16376.639487
125    16452.698971
Name: Predicted_Close, Length: 126, dtype: float64

In [ ]:
mdft["Change"]=(mdft["Predicted_Close"]-mdft["Open"])*100/mdft["Open"]
mdft["Label"] = np.where(mdft["Change"] >= 0, 1, 0)


In [ ]:
mdft.head()

,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,High,Low,Volume,t,sine_pred,sin_5,cos_5,Predicted_Close,Change,Label
0,2016-01-04,saudi arabia break tie iran amid row saudi exe...,exclus uk govern urg reveal role get saudi ara...,china halt trade stock fall,earthquak measur magnitud strike indi,viru link shrink newborn brain spread rapidli ...,new photo spacex booster sooti undamag rock,indian consul afghanistan attack gunm,iran call major haj,denmark respond swedish border check control e...,...,17405.48047,16957.63086,148060000,1856,0.000988,9.510565e-01,0.309017,16956.851557,-2.577515,0
1,2016-01-05,dutch govern back strong encrypt condemn backdoor,isi milit strap bomb year old boy blow kill ch...,canada proceed controversi billion saudi arm d...,brazil head worst recess,british isi milit s desper plea doctor join te...,malaysia s school start hold anti isi talk kid,saudi arabia omit uk death penalti strategi sa...,isi execut femal journalist post complain limi...,u s justic depart su volkswagen billion allege...,...,17195.83984,17038.60938,105750000,1857,0.000564,5.877853e-01,-0.809017,16897.814172,-1.456106,0
2,2016-01-06,china shut stock market min loss,arm suspect shot dead tri storm pari polic statio,oil price fall barrel fresh year lo,german outrag mayor advic women raft harassmen,german public broadcast zdf apologis delay rep...,iran accus saudi bomb embassi sana yeme,protest demand canada cancel b saudi contrac,snooper s charter cost british live mp warn ns...,islam scholar malaysia indonesia issu fatwa po...,...,17154.83008,16817.61914,120250000,1858,-0.000091,-5.877853e-01,-0.809017,16553.411982,-3.505824,0
3,2016-01-07,report sexual assault women european citi incl...,shelter forcibl convert children christian rai...,iran ban citizen make pilgrimag mecc,gaddafi warn blair oust open door jihadi want ...,turkish firm sell fake life vest soak water re...,german minist say deport possibl cologn sexual...,mao statu china pull author ft gold giant mock...,poacher hunt dog south africa award medal hero...,polic use terror power spi offic blow whistl r...,...,16888.35938,16463.63086,176240000,1859,-0.000375,-9.510565e-01,0.309017,16126.295508,-4.512362,0
4,2016-01-08,el chapo taken prison escap month ag,el chapo caugh,children abus cathol boy choir run pope benedi...,north korea defend latest nuclear test say fat...,group unit nation expert warn chill effect fre...,gunman open hotel popular western tourist hurg...,french propos strip citizenship terror set alarm,hungari support eu sanction poland say hungari...,insid delhi s gadget graveyard west s e wast e...,...,16651.89063,16314.57031,141850000,1860,-0.000027,-2.900657e-13,1.000000,16070.989024,-2.713096,0


In [ ]:
mdft.to_csv("mdft_predictions.csv", index=False)
print("mdft has been saved to 'mdft_predictions.csv'")

mdft has been saved to 'mdft_predictions.csv'


In [ ]:
final_result= mdft[["Date", "Predicted_Close"]]

final_result.to_csv("final_result.csv", index=False)

In [ ]:
# Sort the DataFrame by Date descending (July → January)
final_result = final_result.sort_values("Date", ascending=False).reset_index(drop=True)

# Create submission DataFrame with only required columns
submission = final_result[["Predicted_Close"]].copy()
submission.insert(0, "id", submission.index + 1)  # id = 1,2,3,...126

# Save as CSV
submission.to_csv("sample_submission.csv", index=False)

# Download in Colab
from google.colab import files
files.download("sample_submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
submission = final_result.copy()

# Keep only needed column
submission = submission[["Predicted_Close"]]

# Reverse order (July → Jan)
submission = submission.iloc[::-1].reset_index(drop=True)

# Rename column
submission.rename(columns={"Predicted_Close": "Close"}, inplace=True)

# Add id column
submission.insert(0, "id", submission.index)

# Check shape
print(submission.shape)  # should be (126, 2)

# Save CSV
submission.to_csv("submission.csv", index=False)

(126, 2)


In [ ]:
df = final_result.copy()

# Date column ko datetime me convert karo
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)

# Sirf required columns lo
df = df[["Date", "Predicted_Close"]]

# July → January order (descending date)
df = df.sort_values("Date", ascending=False)

# Date column hata do
df = df.drop(columns=["Date"])

# Column name change karo
df.rename(columns={"Predicted_Close": "Close"}, inplace=True)

# ID column add karo (1 se start)
df.insert(0, "id", range(1, len(df) + 1))

# Check rows
print(len(df))  # should be 126

# Save submission file
df.to_csv("submission.csv", index=False)

126
